In [1]:
import pandas as pd
from sklearn.decomposition import NMF
import numpy as np

# NMF to Predict Movie Ratings

For part four lets import the movie rating data and see how NMF predictions compare to the recommender systems predictions.

In [2]:
# First import all the data
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# and make a function to caclulate rmse
def rmse(pred, real):
    """Calculate the rmse using a modified version of the week 3 function"""
    pred[np.isnan(pred)]=3 #In case there is nan values in prediction, it will impute to 3.
    return np.sqrt(((real-pred)**2).mean())

NMF can’t handle null values so we have to replace the null movie ratings in the train data. Let's start by using 0, then run the NMF model. 

In [3]:
# lets see how many null values there are if I make a table of user ID's as rows and movie IDs as columns
ratings_by_user_df = train.pivot(index='uID', columns='mID', values='rating')
print('there are', sum(ratings_by_user_df.isna().sum()), 'null values.')
# thats a lot of null values. Lets fill them with the average rating
ratings_by_user_df = ratings_by_user_df.fillna(value = 0)
# check it worked
print('there are now', sum(ratings_by_user_df.isna().sum()), 'null values.')

there are 21430414 null values.
there are now 0 null values.


In [4]:
# lets run it through NMF
nmf = NMF(n_components=5, random_state=0,)
# grap the output matrices
w1 = nmf.fit_transform(ratings_by_user_df)
h1 = nmf.components_

In [5]:
# mutiply the matrixes to get a prediction
pred = np.matmul(w1, h1)
# turn it in to a dataframe
df_pred = pd.DataFrame(data = pred, index = ratings_by_user_df.index.values, columns = ratings_by_user_df.columns.values) 
# assign a columns for user id
df_pred['uID'] = df_pred.index.values
# use melt to change it from wide to long format
df_pred_long = pd.melt(df_pred, id_vars=['uID'], var_name='mID', value_name='predicted_rating')
# merge it ith the actual ratings
df_pred_vs_test = df_pred_long.merge(test, on=['uID', 'mID'])
# and lets see how the reating compare!
df_pred_vs_test

,uID,mID,predicted_rating,rating
0,6,1,0.548484,4
1,8,1,0.935704,4
2,21,1,0.141737,3
3,23,1,1.976801,4
4,26,1,1.890181,3
...,...,...,...,...
300001,5333,3952,0.893497,4
300002,5359,3952,0.714152,5
300003,5682,3952,0.739560,3
300004,5812,3952,0.783215,4


In [6]:
rmse(df_pred_vs_test.predicted_rating.to_numpy(), df_pred_vs_test.rating.to_numpy())

2.9914125525829425

As can be seen in the second row of the comparison data frame some of those ratings are significantly off. The baseline “predict to three” function from last week gave a rmse of ~1.26 while this NMF model gave ~3.0. This is because there is no way to tell Sklearn's implementation of NMF to ignore certain values. And since there's more null values (now zeros) than actual ratings in this data that has a large impact on the calculation being run in the NMF function, resulting in much lower ratings than what would be expected.

One way to fix this is just to input a more representative value for null. For example, let's try 3.

In [8]:
# fill the values with 3 insted of 0
ratings_by_user_df = train.pivot(index='uID', columns='mID', values='rating')
ratings_by_user_df = ratings_by_user_df.fillna(value = 3)
# rerun nmf
nmf = NMF(n_components=5, random_state=0,)
w1 = nmf.fit_transform(ratings_by_user_df)
h1 = nmf.components_
pred = np.matmul(w1, h1)

# recalculate the rmse
df_pred = pd.DataFrame(data = pred, index = ratings_by_user_df.index.values, columns = ratings_by_user_df.columns.values) 
df_pred['uID'] = df_pred.index.values
df_pred_long = pd.melt(df_pred, id_vars=['uID'], var_name='mID', value_name='predicted_rating')
df_pred_vs_test = df_pred_long.merge(test, on=['uID', 'mID'])
df_pred_vs_test

,uID,mID,predicted_rating,rating
0,6,1,3.168856,4
1,8,1,3.274621,4
2,21,1,3.026765,3
3,23,1,3.254364,4
4,26,1,3.177183,3
...,...,...,...,...
300001,5333,3952,3.129424,4
300002,5359,3952,3.162005,5
300003,5682,3952,3.161739,3
300004,5812,3952,3.145391,4


In [9]:
rmse(df_pred_vs_test.predicted_rating.to_numpy(), df_pred_vs_test.rating.to_numpy())

1.144331431727138

Now we can see that the predicted rating more closely matches the actual rating. In fact NMF is outperforming the baseline recommendation system. One way to improve Sklearn’s NMF function would be to add an option to set null values to the average.